<a href="https://colab.research.google.com/github/Fazira2025/weatherwise-harahap-zizi/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub



## 📦 Setup and Configuration
Import required packages and setup environment.

In [5]:
# Add any other setup code here
import ipywidgets as widgets
from IPython.display import display, clear_output



## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    pass

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass


In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    pass

## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    pass

## 🧭 User Interface

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Header display (blue color using HTML)
header_html = widgets.HTML(
    value="<pre style='color:blue; font-weight:bold;'>"
          "============================================\n"
          "               Weather Wise                 \n"
          "============================================"
          "</pre>"
)

# Inputs for user info
name_input = widgets.Text(description='Name:', placeholder='Enter your name')
location_input = widgets.Text(description='Location:', placeholder='Enter your location')
check_button = widgets.Button(description='Check')

output = widgets.Output()

# Container for name/location inputs and check button
input_box = widgets.VBox([name_input, location_input, check_button])

# Container for main menu input and button (starts hidden)
menu_choice_input = widgets.Text(description='Choice:', placeholder='Enter 1-5')
menu_submit_button = widgets.Button(description='Submit')
menu_input_box = widgets.HBox([menu_choice_input, menu_submit_button])
menu_input_box.layout.display = 'none'  # hide initially

# Back button for submenus (hidden initially)
back_button = widgets.Button(description='Back to Main Menu')
back_button.layout.display = 'none'

def show_menu(name, location):
    with output:
        clear_output()
        print(f"Welcome, {name} what will you check about {location}!\n")
        print("Main Menu:")
        print("1. Check Current Weather & View Weather Forecast")
        print("2. Check Current Weather Hourly")
        print("3. Activities Suggestions & Weather Prediction assistant")
        print("4. Check Other Location")
        print("5. Exit Program")
    menu_input_box.layout.display = ''  # show menu input box
    back_button.layout.display = 'none'  # hide back button in main menu

def show_option_1(name, location):
    with output:
        clear_output()
        print(f"Option 1 Selected: Current Weather & Forecast for {location}")
        print("Here you would show the weather forecast details...\n")
    menu_input_box.layout.display = 'none'  # hide menu choice input
    back_button.layout.display = ''  # show back button

def on_check_clicked(b):
    name = name_input.value.strip()
    location = location_input.value.strip()
    if not name or not location:
        with output:
            clear_output()
            print("Please enter both your name and location.")
        return

    # Hide user info inputs and button
    input_box.layout.display = 'none'
    show_menu(name, location)

def on_menu_submit_clicked(b):
    choice = menu_choice_input.value.strip()
    name = name_input.value.strip()
    location = location_input.value.strip()
    if choice not in ['1','2','3','4','5']:
        with output:
            clear_output()
            print("Invalid choice. Please enter a number between 1 and 5.\n")
            show_menu(name, location)
        return
    if choice == '5':
        with output:
            clear_output()
            print(f"Thank you for using Weather Wise. Goodbye! {name}")
        menu_input_box.layout.display = 'none'
        back_button.layout.display = 'none'
        return
    if choice == '1':
        show_option_1(name, location)
    else:
        with output:
            clear_output()
            print(f"You selected option {choice}. (Functionality not implemented yet)")
            print("\n(Main menu will be shown again)")
        show_menu(name, location)
        menu_choice_input.value = ''

def on_back_clicked(b):
    name = name_input.value.strip()
    location = location_input.value.strip()
    show_menu(name, location)

check_button.on_click(on_check_clicked)
menu_submit_button.on_click(on_menu_submit_clicked)
back_button.on_click(on_back_clicked)

# Display all widgets
display(header_html, input_box, output, menu_input_box, back_button)

HTML(value="<pre style='color:blue; font-weight:bold;'>============================================\n         …

Output()

Button(description='Back to Main Menu', layout=Layout(display='none'), style=ButtonStyle())

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.